# Rasterize Natural Gas Pipeline Network
Creates a raster version at the same extent and cell size of the MIT cost surface.

In [ ]:
import rasterio
from rasterio import features
import geopandas as gpd
import numpy as np

In [ ]:
#Load the cost surface raster
cost_surface_dataset = rasterio.open('../data/costsurface500m.img')
#Get the surface 
cost_surface = cost_surface_dataset.read(1)
#Extract the coordinate reference system
cost_crs = cost_surface_dataset.crs

In [ ]:
#Read in features
pipelines_gdb = 'C:\\Workspace\\Rextag_7May2020\\Rextag_Zone2_Natural_Gas.gdb'
pipelines_lyr = 'NaturalGasPipelines'

In [ ]:
#Read in pipeline dataset as a geodataframe
gdf = gpd.read_file(pipelines_gdb,driver='FileGDB',layer=pipelines_lyr)

In [ ]:
#Transform to same crs as cost surface
gdf_nc = gdf.to_crs(cost_crs)

In [ ]:
#Convert features to raster
image = features.rasterize(shapes=gdf_nc.geometry,
                           out_shape=cost_surface_dataset.shape,
                           transform=cost_surface_dataset.transform,
                           all_touched=False)

In [ ]:
#Update metadata for output
cost_meta.update(
    {'driver':'GTiff',
     'dtype':'uint8',
     'nodata':0,
    })

In [ ]:
#Export as tiff
with rasterio.open('..\\data\\processed\\USA_pipes.tif','w',**cost_meta) as dst:
    dst.write(image, indexes=1)
    


#### Subset to extent of `costsurface_subset.tif`

In [ ]:
#Read in subset raster
ds_cost_subset = rasterio.open('../data/costsurface_subset.tif')
arr_cost = ds_cost_subset.read(1)
arr_cost.shape

In [ ]:
#Re-open tiff
ds_pipes_all = rasterio.open('..\\data\\processed\\USA_pipes.tif')

In [ ]:
#Get the bounding box
from shapely.geometry import box
bounding_box = ds_cost_subset.bounds
print(bounding_box)
bbox = box(*bounding_box)

In [ ]:
#Mask the data
import rasterio.mask
out_image, out_transform = rasterio.mask.mask(
    dataset=ds_pipes_all,
    shapes=[bbox],
    all_touched=True,
    nodata=0,
    crop=True,
)
#Drop last row
out_image_fixed = out_image[:,0:-1,:]
out_image_fixed.shape

In [ ]:
#Update the image
out_meta = ds_cost_subset.meta
out_meta.update({"driver": "GTiff",
                 "dtype":"uint8",
                 "height": out_image_fixed.shape[1],
                 "width": out_image_fixed.shape[2],
                 "transform": out_transform})
with rasterio.open("../data/processed/subset_pipes2.tif", "w", **out_meta) as dest:
    dest.write(out_image)
    